## Naive Bayes

O Teorema de Bayes diz que:

$$P(A|B) = \frac{P(B|A) \times P(A)}{P(B)}$$

Expandindo para um caso mais prático, suponha que $y$ seja a variável resposta com $i$ possibilidades e $x$ conjunto de $j$ variáveis explicativas, sendo elas $x_1, x_2, ..., x_j$. Logo,

$$P(y_i|x) = \frac{P(x|y_i) \times P(y_i)}{P(x)}$$

Como $P(x)$ é igual para todos as possibilidades de $y$, podemos ignorar o denominador da equação. Assumindo que todos os atributos são *independentes*, $P(x|y_i)$ pode ser decomposto em $P(x_1|y_i) \times P(x_2|y_i) \times ... \times P(x_j|y_i)$. Logo, a probabilidade de um exemplo pertencer à classe $y_i$ é proporcional à expressão

$$P(y_i|x) \propto P(y_i) \prod P(x_j|y_i)$$

Logo, podemos dizer que a predição $\hat{y}$ será

$$\hat{y} = \underset{y_i}{\operatorname{arg max}}  P(y_i|x)$$

Ilustrando no exemplo de spam:
    
Fazemos um histograma de ocorrências de palavras:

- Não spam (N) - 8 mensagens:
   - Prezado: 8
   - Sr.: 5
   - Café: 3
   - Dinheiro: 1
- Spam (S) - 4 mensagens:
   - Prezado: 2
   - Sr.: 1
   - Café: 0
   - Dinheiro: 4

**Para pensar:** Quais as probabilidades de cada palavra dado o evento N? E dado o evento S?

**Para pensar:** Se recebermos uma mensagem com "Prezado Sr.", como saber se é spam ou não?

- Passo 1: Probabilidade a priori
- Passo 2: Probabilidade a posteriori
- Passo 3: Multiplicação

$$P(y_i|x) \propto P(y_i) \prod P(x_j|y_i)$$

In [ ]:
P(N | Prezado Sr) = (0.67) * (0.47 * 0.29)
P(S | Prezado Sr) = (0.33) * (0.29 * 0.14)

In [1]:
N = (0.67) * (0.47 * 0.29)

In [2]:
S = (0.33) * (0.29 * 0.14)

In [3]:
N > S

True

Logo, concluímos que a mensagem não é spam. 

**Para pensar:** Se recebermos uma mensagem com "Dinheiro Dinheiro Dinheiro", como saber se é spam ou não?

In [ ]:
P(N | Dinheiro Dinheiro Dinheiro) = (0.67) * (0.06 * 0.06 * 0.06)
P(S | Dinheiro Dinheiro Dinheiro) = (0.33) * (0.57 * 0.57 * 0.57)

In [4]:
N = (0.67) * (0.06 * 0.06 * 0.06)

In [5]:
S = (0.33) * (0.57 * 0.57 * 0.57)

In [13]:
S > N

True

Logo, concluímos que a mensagem é spam. 

Podemos também ir além e normalizar, para obter o resultado em termos de probabilidade:

In [13]:
N = (0.67) * (0.06 * 0.06 * 0.06)
S = (0.33) * (0.57 * 0.57 * 0.57)

In [14]:
P_n = N / (N + S)
P_s = S / (N + S)

In [15]:
P_n

0.0023624511311997817

In [16]:
P_s

0.9976375488688002

### Implementação no sklearn

In [1]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB

In [2]:
df = pd.DataFrame({
    'prezado':  [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1],
    'sr':       [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0],
    'cafe':     [0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    'dinheiro': [0, 0, 0, 0, 1, 0, 0, 0, 1, 2, 1, 0],
    'spam':     [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
})

df

,prezado,sr,cafe,dinheiro,spam
0,1,0,0,0,0
1,1,1,0,0,0
2,1,1,2,0,0
3,1,1,0,0,0
4,1,1,1,1,0
5,1,1,0,0,0
6,1,0,0,0,0
7,1,0,0,0,0
8,0,0,0,1,1
9,0,0,0,2,1


In [3]:
x_treino = df.loc[:, :'dinheiro']
y_treino = df.loc[:, 'spam']

modelo = MultinomialNB()

modelo.fit(x_treino, y_treino)

MultinomialNB()

In [4]:
nova_instancia = pd.DataFrame({
    'prezado': [1],
    'sr': [1],
    'cafe': [0],
    'dinheiro': [0],
})

nova_instancia

,prezado,sr,cafe,dinheiro
0,1,1,0,0


In [5]:
modelo.predict_proba(nova_instancia)

array([[0.83161512, 0.16838488]])

Os resultados das probabilidades foram diferentes do que calculamos. Vamos analisar outro caso, primeiro com cálculos a mão e em seguida usando o sklearn.

$$\textrm{P(N | Café Dinheiro Dinheiro Dinheiro)} = 0.67 \times 0.18 \times 0.06 \times 0.06 \times 0.06 = 0.000026$$

$$\textrm{P(S | Café Dinheiro Dinheiro Dinheiro)} = 0.33 \times 0.00 \times 0.57 \times 0.57 \times 0.57 = 0.000000$$

Observe que por mais que tenhamos a palavra dinheiro aparecendo 3 vezes, a probabilidade de ser spam continua sendo 0. Isso acontece porque a palavra café está contida na mensagem, e por não ter nenhuma mensagem que seja spam que tem essa palavra, a probabilidade vai pra zero, zerando o resultado. Como será que está no modelo?

Faz sentido! A chance de ser spam é alta. Isso acontece porque a implementação do sklearn faz uso de uma técnica que adiciona 1 a todos os elementos, pra evitar que um deles possa zerar alguma equação. Dessa forma, as contagens ficam:

- Não spam (N) - 8 mensagens:
   - Prezado: 9
   - Sr.: 6
   - Café: 4
   - Dinheiro: 2
- Spam (S) - 4 mensagens:
   - Prezado: 3
   - Sr.: 2
   - Café: 1
   - Dinheiro: 5

**Para pensar:** As probabilidades de cada palavra dado o evento N muda? E dado o evento S? Se sim, qual o novo valor?

$$\textrm{P(Prezado | N)} = 0.43$$

$$\textrm{P(Sr. | N)} = 0.29$$

$$\textrm{P(Café | N)} = 0.19$$

$$\textrm{P(Dinheiro | N)} = 0.09$$

$$\textrm{P(Prezado | S)} = 0.27$$

$$\textrm{P(Sr. | S)} = 0.18$$

$$\textrm{P(Café | S)} = 0.09$$

$$\textrm{P(Dinheiro | S)} = 0.45$$


**Para pensar:** Qual a probabilidade de ser spam dado que a mensagem é "Café Dinheiro Dinheiro Dinheiro"?

Os números não batem na vírgula. Isso é esperado, acontece porque fizemos uma série de aproximações, mas agora faz mais sentido. Tomando esse fator como step usado no sklearn, podemos recalcular a probabilidade que tínhamos achado anteriormente:


$$\textrm{P(N | Prezado Sr.)} \propto \textrm{P(N) x P(Prezado | N) x P(Sr. | N)} = 0.67 x 0.43 x 0.29 = 0.083$$

$$\textrm{P(S | Prezado Sr.)} \propto \textrm{P(S) x P(Prezado | S) x P(Sr. | S)} = 0.33 x 0.27 x 0.18 = 0.016$$

Normalizando:

### Outros Naive Bayes

In [ ]:
df = pd.DataFrame([[1 , 0 , 1 , 1],
                   [1 , 1 , 1 , 1],
                   [0 , 1 , 0 , 0],
                   [0 , 1 , 1 , 0],
                   [1 , 0 , 0 , 0],
                   [0 , 1 , 0 , 0],
                   [0 , 0 , 1 , 0],
                   [0 , 1 , 0 , 0],
                   [1 , 0 , 0 , 0],
                   [0 , 0 , 0 , 0],
                   [0 , 1 , 0 , 0],
                   [0 , 0 , 1 , 0]], columns = ['MA', 'MD', 'PG', 'F'])

In [ ]:
MA: Mensagem Anonima
MD: Multiplo Destinatario
PG: Provedor Gratuito
F:  Spam ou não

In [ ]:
x_treino = df.loc[:, :'PG']
y_treino = df.loc[:, 'F']

modelo = MultinomialNB()
modelo.fit(x_treino, y_treino)

In [ ]:
nova_instancia = pd.DataFrame({
    'MA': [1],
    'MD': [1],
    'PG': [1]
})

modelo.predict_proba(nova_instancia)

In [ ]:
from sklearn.naive_bayes import BernoulliNB

In [ ]:
x_treino = df.loc[:, :'PG']
y_treino = df.loc[:, 'F']

modelo = BernoulliNB()
modelo.fit(x_treino, y_treino)

In [ ]:
nova_instancia = pd.DataFrame({
    'MA': [1],
    'MD': [1],
    'PG': [1]
})

modelo.predict_proba(nova_instancia)

### Exercício Predição de Spam

**Objetivo:** Fazer a predição de spam ou não spam no dataset de spam_ham.

[Link para download do dataset](https://www.dropbox.com/s/h9wxw3ak6fq7bcs/spam_ham.csv?dl=0)
    
**Passo 1:** Preprocessamento
- Transformar todas as strings em maiúsculo ou minúsculo
- Remover pontuação
- Realizar o split (criar tokens) onde tem espaço, criando para cada linha, uma lista de palavras
- Determinar a contagem total de todas as palavras - pode usar dicionário
- Escolher as 100 palavras que mais ocorrem
- Para cada mensagem, determinar as ocorrências dessas 100 palavras escolhidas

**Passo 2:** Modelagem
- Escolher qual Naive Bayes utilizaremos
- Fazer o ajuste

**Passo 3:** Avaliação
- Extrair as métricas de avaliação
    - ROC AUC
    - Gini
    - Acurácia
    - Precisão
    - Especificidade
    - ...